In [1]:
import sys
sys.path.append("../")
from PREDICT import PREDICT
from PREDICT.Models import RecalibratePredictions
from PREDICT.Metrics import Accuracy, AUC, F1Score, Precision, Recall, Sensitivity, Specificity
from PREDICT.Triggers import AccuracyThreshold

import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Fake data with stepchange in incidence rate to test recalibration
mydict = {
    'date': list(),
    'outcome': list(),
    'prediction': list()
}
np.random.seed(42)
numweeks = 104
startDate = pd.to_datetime('01-01-2024', dayfirst=True)
endDate = pd.to_datetime('31-12-2024', dayfirst=True)
numdays = (endDate - startDate).days
accuracy_start = 0.9

for i in range(numdays):
    curday = startDate + dt.timedelta(days=i)
    curaccuracy = accuracy_start - 0.5 * np.random.rand()
    # Generate fake outcomes
    curoutcomes = np.random.choice([0, 1], size=100, p=[0.7, 0.3])
    # Generate fake outcomes to match current accuracy level
    correct = np.random.choice([0, 1], size=100, p=[1-curaccuracy, curaccuracy])
    ix = list(np.where(correct==0)[0])
    curpredictions = curoutcomes.copy()
    curpredictions[ix] = 1-curpredictions[ix]
    # Append to dictionary
    mydict['date'].extend([curday]*100)
    mydict['outcome'].extend(curoutcomes)
    mydict['prediction'].extend(curpredictions)

df = pd.DataFrame(mydict)

In [3]:
model = RecalibratePredictions()
model.trigger = AccuracyThreshold(model=model, threshold=0.9)
mytest = PREDICT(data=df, model=model, startDate='min', endDate='max', timestep='week')
mytest.addLogHook(Accuracy(model))
mytest.addLogHook(AUC(model))
mytest.addLogHook(F1Score(model))
mytest.addLogHook(Precision(model))
mytest.addLogHook(Recall(model))
mytest.addLogHook(Sensitivity(model))
mytest.addLogHook(Specificity(model))

In [4]:
mytest.run()

c:\Users\hsssre\AppData\Local\anaconda3\envs\pythonbasic\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


ValueError: Expected a 2-dimensional container but got <class 'pandas.core.series.Series'> instead. Pass a DataFrame containing a single row (i.e. single sample) or a single column (i.e. single feature) instead.

In [5]:
model.postPredictHooks

AttributeError: 'RecalibratePredictions' object has no attribute 'postPredictHooks'